In [13]:
import circlify as circ

data = [19, 17, 13, 11, 7, 5, 3, 2, 1]
circles = circ.circlify(data, with_enclosure=True)


In [12]:
#!/usr/bin/env python
# encoding: utf-8

"""Basic circle packing algorithm based on 2 algorithms.
Circles are first arranged via a version of A1.0 by Huang et al (see
https://home.mis.u-picardie.fr/~cli/Publis/circle.pdf for details) and then
enclosed in a circle created around them using Matoušek-Sharir-Welzl algorithm
used in d3js (see https://beta.observablehq.com/@mbostock/miniball,
http://www.inf.ethz.ch/personal/emo/PublFiles/SubexLinProg_ALG16_96.pdf, and
https://github.com/d3/d3-hierarchy/blob/master/src/pack/enclose.js)
"""

import sys
import math
from math import sqrt, pi
import collections
import itertools
import logging
import random


__version__ = '0.9.1'


try:
    import matplotlib.pyplot as plt
    import matplotlib.patches as pltp

    def bubbles(circles, labels, lim=None):
        """Debugging function displays circles with matplotlib."""
        fig, ax = plt.subplots(figsize=(8.0, 8.0))
        n_missing_labels = len(circles) - len(labels)
        if n_missing_labels > 0:
            labels += [''] * n_missing_labels
        for circle, label in zip(circles, labels):
            x, y, r = circle
            ax.add_patch(pltp.Circle((x, y), r, alpha=0.2,
                                     linewidth=2, fill=False))
            ax.text(x, y, label)
        if lim is None:
            lim = max([max(abs(c.x) + c.r, abs(c.y) + c.r)
                       for c in circles])
        plt.xlim(-lim, lim)
        plt.ylim(-lim, lim)
        plt.show()
except ImportError:
    pass


Circle = collections.namedtuple('Circle', ['x', 'y', 'r'])

log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())


def distance(circle1, circle2):
    """Compute distance between 2 cirlces."""
    x1, y1, r1 = circle1
    x2, y2, r2 = circle2
    x = x2 - x1
    y = y2 - y1
    return sqrt(x * x + y * y) - r1 - r2


def get_intersection(circle1, circle2):
    """Calculate intersections of 2 circles
    Based on https://gist.github.com/xaedes/974535e71009fa8f090e
    Credit to http://stackoverflow.com/a/3349134/798588
    Returns:
        2 pairs of coordinates. Each pair can be None if there are no or just
        one intersection.
    """
    x1, y1, r1 = circle1
    x2, y2, r2 = circle2
    dx, dy = x2 - x1, y2 - y1
    d = sqrt(dx * dx + dy * dy)
    if d > r1 + r2:
        log.debug('no solution, the circles are separate: %s, %s',
                  circle1, circle2)
        return None, None
    if d < abs(r1 - r2):
        log.debug('no solution, circles contained within each other: %s, %s',
                  circle1, circle2)
        return None, None
    if d == 0 and r1 == r2:
        log.debug('no solution, circles are coincident: %s, %s',
                  circle1, circle2)
        return None, None
    a = (r1 * r1 - r2 * r2 + d * d) / (2 * d)
    h = sqrt(r1 * r1 - a * a)
    xm = x1 + a * dx / d
    ym = y1 + a* dy / d
    xs1 = xm + h * dy / d
    xs2 = xm - h * dy / d
    ys1 = ym - h * dx / d
    ys2 = ym + h * dx / d
    if xs1 == xs2 and ys1 == ys2:
        return (xs1, ys1), None
    return (xs1, ys1), (xs2, ys2)


def get_placement_candidates(radius, c1, c2):
    """Generate placement candidates for 2 existing placed circles.
    Args:
        radius: radius of the new circle to place.
        c1: first existing placed circle.
        c2: second existing placed circle.
    Returns:
        A pair of candidate cirlces where one or both value can be None.
    """
    margin = 10.0 * sys.float_info.epsilon
    ic1 = Circle(c1.x, c1.y, c1.r + (radius + margin))
    ic2 = Circle(c2.x, c2.y, c2.r + (radius + margin))
    i1, i2 = get_intersection(ic1, ic2)
    if i1 is None:
        return None, None
    i1_x, i1_y = i1
    candidate1 = Circle(i1_x, i1_y, radius)
    if i2 is None:
        return candidate1, None
    i2_x, i2_y = i2
    candidate2 = Circle(i2_x, i2_y, radius)
    return candidate1, candidate2


def get_hole_degree(candidate, placed_circles, pc1, pc2):
    """Calculate the hole degree of a candidate circle.
    Note pc1 and pc2 must not be used in the evaluation of the minimum
    distance.
    Args:
        candidate: candidate circle.
        placed_circles: sequence of circles already placed.
        pc1: first placed circle used to place the candidate.
        pc2: second placed circle used to place the candidate.
    Returns:
        hole degree defined as (1 - d_min / r_i) where d_min is a minimum
        disance between the candidate and the circles other than the one
        used to place the candidate and r_i the radius of the candidate.
    """
    #min_dist = None
    lsq = 0.
    for pc in placed_circles:
        if pc1 is not None and pc1 == pc:
            continue
        if pc2 is not None and pc2 == pc:
            continue
        lsq += sqrt((candidate.y - pc.y) ** 2.0 + (candidate.x - pc.x) ** 2.0)
        #if min_dist is None or min_dist > dist:
            #min_dist = dist
    #if min_dist is None:
        #return 0.0
    return -sqrt(lsq)


def pack_A1_0(data):
    """Pack circles whose radius is linked to the input data.
    This algorithm is based on the Huang et al. heuristic.
    Args:
        data: sorted (descending) list of value to circlify.
    Returns:
        list of circlify.Circle.
    """
    assert data == sorted(data, reverse=True), 'data must be sorted (desc)'
    placed_circles = []
    for value in data:
        radius = sqrt(value)
        n_circles = len(placed_circles)
        # Place first 2 circles on each side of (0, 0)
        if n_circles <= 1:
            x = radius if n_circles == 0 else -radius
            circle = Circle(x, 0.0, radius)
            placed_circles.append(circle)
            continue
        mhd = None
        lead_candidate = None
        for (c1, c2) in itertools.combinations(placed_circles, 2):
            for cand in get_placement_candidates(radius, c1, c2):
                if cand is not None:
                    # Ignore candidates that overlap with any placed circle.
                    overlap = False
                    for pc in placed_circles:
                        if distance(pc, cand) < 0.0:
                            overlap = True
                            break
                    if overlap:
                        continue
                    hd = get_hole_degree(cand, placed_circles, c1, c2)
                    if mhd is None or hd > mhd:
                        mhd = hd
                        lead_candidate = cand
        if lead_candidate is None:
            log.info('cannot place circle for all values')
            break
        placed_circles.append(lead_candidate)
    return placed_circles


def extendBasis(B, p):
    """Extend basis to ...  """
    if enclosesWeakAll(p, B):
        return [p];

    # If we get here then B must have at least one element.
    for i in range(len(B)):
        if enclosesNot(p, B[i]) and enclosesWeakAll(encloseBasis2(B[i], p), B):
            return [B[i], p];

    # If we get here then B must have at least two elements.
    for i in range(len(B) - 1):
        for j in range(i + 1, len(B)):
            if enclosesNot(encloseBasis2(B[i], B[j]), p) and \
                    enclosesNot(encloseBasis2(B[i], p), B[j]) and \
                    enclosesNot(encloseBasis2(B[j], p), B[i]) and \
                    enclosesWeakAll(encloseBasis3(B[i], B[j], p), B):
                return [B[i], B[j], p];
    raise RuntimeError('If we get here then something is very wrong')


def enclosesNot(a, b):
    dr = a.r - b.r
    dx = b.x - a.x
    dy = b.y - a.y
    return dr < 0 or dr * dr < dx * dx + dy * dy


def enclosesWeak(a, b):
    dr = a.r - b.r + 1e-6
    dx = b.x - a.x
    dy = b.y - a.y
    return dr > 0 and dr * dr > dx * dx + dy * dy


def enclosesWeakAll(a, B):
    for i in range(len(B)):
        if not enclosesWeak(a, B[i]):
            return False
    return True


def encloseBasis(B):
    if len(B) == 1:
        return B[0]
    elif len(B) == 2:
        return encloseBasis2(B[0], B[1])
    else:
        return encloseBasis3(B[0], B[1], B[2])


def encloseBasis2(a, b):
    x1, y1, r1 = a.x, a.y, a.r
    x2, y2, r2 = b.x, b.y, b.r
    x21 = x2 - x1
    y21 = y2 - y1
    r21 = r2 - r1
    l = sqrt(x21 * x21 + y21 * y21);
    return Circle((x1 + x2 + x21 / l * r21) / 2,
                  (y1 + y2 + y21 / l * r21) / 2,
                  (l + r1 + r2) / 2)


def encloseBasis3(a, b, c):
    x1, y1, r1 = a.x, a.y, a.r
    x2, y2, r2 = b.x, b.y, b.r
    x3, y3, r3 = c.x, c.y, c.r
    a2 = x1 - x2
    a3 = x1 - x3
    b2 = y1 - y2
    b3 = y1 - y3
    c2 = r2 - r1
    c3 = r3 - r1
    d1 = x1 * x1 + y1 * y1 - r1 * r1
    d2 = d1 - x2 * x2 - y2 * y2 + r2 * r2
    d3 = d1 - x3 * x3 - y3 * y3 + r3 * r3
    ab = a3 * b2 - a2 * b3
    xa = (b2 * d3 - b3 * d2) / (ab * 2) - x1
    xb = (b3 * c2 - b2 * c3) / ab
    ya = (a3 * d2 - a2 * d3) / (ab * 2) - y1
    yb = (a2 * c3 - a3 * c2) / ab
    A = xb * xb + yb * yb - 1
    B = 2 * (r1 + xa * xb + ya * yb)
    C = xa * xa + ya * ya - r1 * r1
    if A != 0.0:
        r = -(B + sqrt(B * B - 4 * A * C)) / (2 * A)
    else:
        r = -C / B
    return Circle(x1 + xa + xb * r, y1 + ya + yb * r, r)


def scale(circles, enclosure, target):
    """Scale circles in enclosure to fit in the target circle.
    Args:
        circles: Circle objects to scale.
        enclusure: Circle that contains all circles.
        target: target Circle to scale to.
    Returns:
        scaled circles
    """
    r = target.r / enclosure.r
    dx = target.x - enclosure.x
    dy = target.y - enclosure.y
    scaled = []
    for circle in circles:
        x_c, y_c, r_c = circle
        scaled.append(Circle((x_c + dx) * r, (y_c + dy) * r, r_c * r))
    return scaled


def enclose(circles):
    """Shamelessly adapted from d3js.
    See https://github.com/d3/d3-hierarchy/blob/master/src/pack/enclose.js
    """
    B = []
    p, e = None, None
    #random.shuffle(circles)

    n = len(circles)
    i = 0
    while i < n:
        p = circles[i];
        if e is not None and enclosesWeak(e, p):
            i = i + 1
        else:
            B = extendBasis(B, p)
            e = encloseBasis(B)
            i = 0
    return e


def circlify(data, target_enclosure=None, with_enclosure=False):
    """Pack and enclose circles whose radius is linked to the input data.
    Args:
        data: sorted (descending) array of values.
        target_enclosure: target ciriclify.Circle where circles should fit in.
        with_enclosure: appends the target circle to the output if True.
    Returns:
        list of circligy.Circle as value for element of data.
    """
    packed = pack_A1_0(data)
    enclosure = enclose(packed)
    if target_enclosure is None:
        target_enclosure = Circle(0.0, 0.0, 1.0)
    if enclosure is None:
        return packed
    packed_and_scaled = scale(packed, enclosure, target_enclosure)
    if with_enclosure:
        packed_and_scaled.append(target_enclosure)
    return packed_and_scaled

In [28]:
a= circlify(data)

In [29]:
enclose(a)

Circle(x=-1.3877787807814457e-17, y=-6.938893903907228e-17, r=0.9999999999999999)

In [25]:
#!/usr/bin/env python
# encoding: utf-8

"""Tests for circlify"""


from circlify import Circle, circlify


# Set this variable to True to get a display of the layout (req matlplotlib)
display_layout = False


class SpecialCases(unittest.TestCase):
    """Hedge cases and obvious answers handling."""

    def test_empty_input(self):
        """What do we do when the output is empty?"""
        actual = circlify([])
        self.assertEqual(actual, [])

    def test_single_value(self):
        """If there is only one value, it should occupy the whole circle."""
        actual = circlify([2.0])
        expected = [Circle(0.0, 0.0, 1.0)]
        self.assertEqual(actual, expected)

    def test_two_equal_values(self):
        """Two equal circle cases is also trivial."""
        # Force scaling to .5 so that each circle radius is brought to 0.5.
        actual = circlify([1.0, 1.0])
        expected = [Circle(0.5, 0.0, 0.5), Circle(-0.5, 0.0, 0.5)]
        self.assertEqual(actual, expected)


class TestCaseWithDisplay(unittest.TestCase):
    """Display the result of the placement of the circle."""

    def display(self, circles, labels):
        """Forwards call to circlify.bubbles()."""
        try:
            if display_layout:
                from circlify import bubbles
                bubbles(circles, labels)
        except AttributeError as err:
            log.error("%s. %s", err, "Did you install matplotlib?")


class PrimeSerieTestCase(TestCaseWithDisplay):
    """Consider a simple sequence of prime number for radius to our circles."""

    def setUp(self):
        """Sets up the primes sequence 1, 2, 3, ... up to 19."""
        self.data = [19, 17, 13, 11, 7, 5, 3, 2, 1]

    def test_circlify(self):
        """Check the coordinates of the circles returned are expected."""
        actual = circlify(self.data, with_enclosure=True)
        expected = [Circle(x=0.35776879346704843, y=-0.13064957525245907,
                           r=0.39529216048201216),
                    Circle(x=-0.411432317820337, y=-0.13064957525245907,
                           r=0.3739089508053733),
                    Circle(x=-0.04661299415374866, y=0.4678014425767657,
                           r=0.32697389223002427),
                    Circle(x=-0.045884607890591435, y=-0.6977206243364218,
                           r=0.3007722353441051),
                    Circle(x=-0.6132109517981927, y=0.4490810687795324,
                           r=0.23993324126007678),
                    Circle(x=0.48296614887228806, y=0.4541723195782383,
                           r=0.20278059970175755),
                    Circle(x=0.3252787490004198, y=0.7776370388468007,
                           r=0.15707317711577193),
                    Circle(x=-0.40283175658099674, y=0.7512387781681531,
                           r=0.12824971207048294),
                    Circle(x=0.09222041925800777, y=0.8617116738294696,
                           r=0.09068624109026069),
                    Circle(x=0.0, y=0.0, r=1.0)]
        self.display(actual, [str(v) for v in self.data])
        self.assertEqual(actual, expected)


class CountSerieTestCase(TestCaseWithDisplay):
    """Consider a simple sequence of number for radius to our circles."""

    def setUp(self):
        """Sets up the primes sequence 1, 2, ..."""
        self.data = list(range(7, 1, -1))

    def test_circlify(self):
        """Check the coordinates of the circles returned are expected."""
        actual = circlify(self.data, with_enclosure=True)
        expected = [Circle(x=0.5824456027453089, y=-0.08515409741642607,
                           r=0.41136250504733196),
                    Circle(x=-0.20976457776763055, y=-0.08515409741642607,
                           r=0.3808476754656075),
                    Circle(x=0.15769153632817096, y=0.5438978793053209,
                           r=0.34766477137653345),
                    Circle(x=0.15910532107887837, y=-0.6704181394216174,
                           r=0.31096082487194077),
                    Circle(x=-0.4586184780594718, y=0.5154819840108337,
                           r=0.2692999739208646),
                    Circle(x=-0.7680630545906644, y=0.13661056172475666,
                           r=0.21988250795031175),
                    Circle(x=0.0, y=0.0, r=1.0)]
        self.display(actual, [str(v) for v in self.data])
        self.assertEqual(actual, expected)


class GeometricSerieTestCase(TestCaseWithDisplay):
    """Consider a simple sequence of number for radius to our circles."""

    def setUp(self):
        """Sets up the primes sequence 1, 2, ..."""
        self.data = sorted([2 ** n for n in range(4, 12)], reverse=True)

    def test_circlify(self):
        """Check the coordinates of the circles returned are expected."""
        actual = circlify(self.data, with_enclosure=True)
        self.display(actual, [str(v) for v in self.data])
        expected = [Circle(x=0.4142135623730951, y=0.0, r=0.5857864376269051),
                    Circle(x=-0.5857864376269051, y=0.0, r=0.4142135623730951),
                    Circle(x=-0.2218254069479773, y=0.6062444788590926,
                           r=0.29289321881345254),
                    Circle(x=-0.20710678118654763, y=-0.49258571550470814,
                           r=0.20710678118654754),
                    Circle(x=0.10281914590763144, y=-0.662720719883036,
                           r=0.14644660940672627),
                    Circle(x=-0.11312522101671703, y=-0.7886890904910677,
                           r=0.10355339059327377),
                    Circle(x=0.041837742530372556, y=-0.8737565926802316,
                           r=0.07322330470336313),
                    Circle(x=-0.18045635173699437, y=-0.22990093891844118,
                           r=0.051776695296636886),
                    Circle(x=0.0, y=0.0, r=1.0)]
        self.assertEqual(actual, expected)


if __name__ == '__main__':
    import logging
    logging.basicConfig(level='INFO')
    unittest.main()

E
ERROR: /Users/owner/Library/Jupyter/runtime/kernel-d378b990-474d-46a6-82b8-9431449ba85c (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/owner/Library/Jupyter/runtime/kernel-d378b990-474d-46a6-82b8-9431449ba85c'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
